<a id='root'></a>
PyUltraLight 2.21.1 Truncated Main General Development Notebook

(12 May 2021)

* Y. Wang: [yourong.f.wang@auckland.ac.nz](mailto:yourong.f.wang@auckland.ac.nz)
* R. Easther
***



<a id='init'></a>
# Initialization and Program Settings

## Loading Packages

In [ ]:
###### Do not touch
MinVersion = 20

import PyUltraLight2 as PyUL

if (PyUL.S_version < MinVersion):
    raise RuntimeError("You need the latest PyULN!")


import numpy as np

np.set_printoptions(suppress=True)

import math

import numba
import numexpr as ne
import time
import pyfftw
import os
import sys
import multiprocessing
import numpy

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec
import matplotlib.animation
from matplotlib.offsetbox import AnchoredText

import scipy.fft

import h5py
import IPython
from IPython.core.display import clear_output, display, Video

from numpy import sqrt, exp, log, log10, sin, cos, tan, pi

G_phi = (sqrt(5)-1)/2

%reload_ext autoreload
%autoreload 2


# Useful Aux Functions
ToCode = PyUL.convert
ToPhys = PyUL.convert_back
CB = PyUL.convert_between

printU = PyUL.printU

save_path = '2021_Paper1'

plt.style.use('default')

plt.rcParams['font.family'] = 'DejaVu Serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rcParams["text.usetex"]= False
plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.facecolor'] = 'w'

from matplotlib.colors import LinearSegmentedColormap
Palette = 'magma'
cyccol=['#000000', '#ffffff','#000000']
divcol=['#003262','#005b96','#ffffff','#d5a756','#B51700']
divcols=['#005b96','#ffffff','#d5a756']

CycPalette = LinearSegmentedColormap.from_list('myCyc', cyccol)

DivPalette  = LinearSegmentedColormap.from_list('myDiv', divcol)
DivPaletteS  = LinearSegmentedColormap.from_list('myDiv', divcols)

EFigSize = (20,12)
EFigSizeMono = (20,6)
VideoSize = (8,8)
FPS = 10
DPI = 72

FrameCap = 60 # Maximum Number of Frames to Animate

########################################

# BH Trajectory

Colors = np.array([[209,17,65],[0,177,89],[0,174,219],[243,119,53],[255,196,37]])/255

# NBody
ENStyle = 'o-'
ENColor = '#ff6c0e' # NBody

EKColor = '#8dd9e7'
EPColor = '#fd8dc8'

# ULDM
EUStyle = 'x-'
EUColor = '#7f1717' # ULDM

EQColor = '#ccb603' # Kinetic 
ERColor = '#6cc25c' # Self-Interaction
EGColor = '#2c8899' # Particle Potential

# Totals
ETStyle = '-'
ETColor = '#080808'

# Auxiliary 
HelperStyle = 'k.'
EmphColor = 'black'
GeomColor = 'orange'

## Loading Data and Config
<a id='Load'></a>

In [ ]:
TimeStamp = PyUL.Runs(save_path)

clear_output()

loc = './' + save_path + '/' + TimeStamp
# Reload Config.. Why don't I use a class?
NS, length, length_units, resol, duration, duration_units, step_factor, save_number, save_options, save_format, s_mass_unit, s_position_unit, s_velocity_unit, solitons,start_time, m_mass_unit, m_position_unit, m_velocity_unit, particles, embeds, Uniform,Density, density_unit ,a, UVel = PyUL.LoadConfig(loc)

if save_number == -1:
    sim_number = PyUL.ULDStepEst(duration,duration_units,
                                          length,length_units,
                                          resol,step_factor, 
                                          save_number = -1)
else:
    sim_number = save_number
    
# Dict of Run Data
EndNum, Loaded = PyUL.Load_npys(loc,save_options, LowMem = False)

In [ ]:
Load2D = True
Load1D = True

if Load2D:
    phasedata = Loaded['2Phase']
    #phidata  = Loaded['2Grav']
    phidataF = Loaded['2GravF']
    data = Loaded['2Density']

if Load1D:
    #phi1D  = Loaded['1Grav']
    phiF1D = Loaded['1GravF']
    linedata = Loaded['1Density']

TMdata = Loaded['NBody']
graddata = Loaded['DF']

NBo = len(particles)

ToCode = PyUL.convert
ToPhys = PyUL.convert_back
CB = PyUL.convert_between

# Time
durationMyr = CB(duration,duration_units,'Myr','t')

# Space
lengthKpc = CB(length,length_units,'kpc','l')
lengthC = ToCode(length,length_units,'l')

# Mass

MassList_MSol = []
MassListSI = []
for TM in particles:
    MassList_MSol.append(CB(TM[0],m_mass_unit,'M_solar_masses','m'))
    MassListSI.append(CB(TM[0],m_mass_unit,'kg','m'))
    
# Density
DensitySI = CB(Density,density_unit,'kg/m3','d')
DensityC = ToCode(Density,density_unit,'d')

## Misc. Pre-Multipliers

Tp = np.arange(EndNum)
Tp = Tp * durationMyr / (sim_number+1)

# NBody_State Vector

XPre = ToPhys(1,'kpc','l')
VPre = ToPhys(1,'km/s','v')

XPreSI = ToPhys(1,'m','l')
VPreSI = ToPhys(1,'m/s','v')

IArray = np.arange(len(TMdata[0]))

# Dimensionful NBody State

TMDataS = np.array(TMdata)
TMDataSI = np.array(TMdata)

TMDataS[:,IArray % 6 <= 2] *= XPre
TMDataS[:,IArray % 6 >= 3] *= VPre

TMDataSI[:,IArray % 6 <= 2] *= XPreSI
TMDataSI[:,IArray % 6 >= 3] *= VPreSI

# Rho data

DPre = CB(1,density_unit,'kg/m3','d')

# Energy
EPre = PyUL.energy_unit

# Field Strength
PPre = PyUL.mass_unit / PyUL.length_unit

print('Unit conversion ready!')


print("==============================================")
print(f"PyULN: Axion Mass Used is {PyUL.axion_E} eV.\n")

print(f"\
1 Mass Unit    = {PyUL.mass_unit:.5g} kg = {PyUL.convert_back(1,'solar_masses','m'):.4g} MSol\n\
1 Length Unit  = {PyUL.length_unit:.5g} m  = {PyUL.convert_back(1,'kpc','l'):.4g} kpc\n\
1 Time Unit    = {PyUL.time_unit:.5g} s  = {PyUL.convert_back(1,'Myr','t')/1000:.4g} Gyr\n\n\
1 Density Unit = {PyUL.mass_unit/PyUL.length_unit**3:.5g} kg/m^3 = {PyUL.mass_unit/PyUL.length_unit**3/PyUL.CritDens} Critical Density\n\
1 Speed Unit   = {PyUL.length_unit/PyUL.time_unit:.5g} m/s\n\
1 Energy Unit  = {EPre:.5g} Joules\n")

Loaded = {}

## 2D Trajectory Plot

In [ ]:
Plot_ULD = True
# Number of ULDM Slices IN BETWEEN start and end (which are always plotted).
NSlices = EndNum//50
Zoom2D = 1

if NSlices >= EndNum:
    
    NSlices = EndNum

try:
    TSD = TimeStamp
except NameError:
    TimeStamp = 'Debug'

plt.clf()

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111)

Boundary = lengthKpc/(2*Zoom2D)

plt.xlim([-Boundary,Boundary])
plt.ylim([-Boundary,Boundary])

ax.set_aspect('equal', adjustable='box')


TMx = TMDataS[:,1::6]
TMy = TMDataS[:,0::6]

plt.scatter([TMx],[TMy])


if Plot_ULD:
    
    planemax = np.max(data)
    planemin = np.min(data)

    levels = np.linspace(planemin, planemax, int(resol/16))

    PlotRange = np.linspace(-lengthKpc/2, lengthKpc/2,resol,endpoint = False)
    
    plt.contour(PlotRange,PlotRange,data[1], levels=levels,cmap = Palette)
    plt.contour(PlotRange,PlotRange,data[EndNum-1], levels=levels,cmap = Palette)


    if NSlices != 0 and EndNum >= NSlices:

        Blink = EndNum/(NSlices+1)

        Index = np.linspace(Blink,EndNum-Blink,NSlices)

        for Ind in Index:
            j = int(Ind)
            plt.contour(PlotRange,PlotRange,data[j], levels=levels,cmap = Palette)
   
ax.grid(True)

Info = '\n'.join((
    TimeStamp,
    r'Number of Bodies: $%.0f$' % (NBo, ),
    r'Resolution: $%.0f^3$' % (resol, ),
    r'Box Length: %.3g kpc' % (lengthKpc, ),
    r'Simulation Time Length: %.3g Myr' % (durationMyr, ),
    r'Saved Snapshots: %.0f (%.0f Plotted)' % (EndNum,NSlices + 2)
    ))

# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

# place a text box in bottom left in axes coords
ax.text(0.01, 0.01, Info, transform=ax.transAxes, fontsize=11,
        verticalalignment='bottom', bbox=props)

if Zoom2D != 1:
    
    props2 = dict(boxstyle='round', facecolor='lightgreen', alpha=0.5)

    Zinfo = f"Zoom Level: {Zoom2D}x"
    ax.text(0.82, 0.95, Zinfo, transform=ax.transAxes, fontsize=12,
            verticalalignment='bottom', bbox=props2)
    
ax.set_ylabel('$y$ / kpc')
ax.set_xlabel('$x$ / kpc')

TrajName = '{}{}{}{}{}'.format("./",save_path,"/_",TimeStamp,'.jpg')
plt.savefig(TrajName, format='jpg', dpi=72)

plt.show()

## Energy

In [ ]:
Relative = True # For Dynamical Friction Cases Only

EFigSize = (10,6.18)
EFigSize_Mono = (10,5)

Snap = False
loc = save_path + '/' + TimeStamp

plt.clf()

#==================================================================================================
## Naming
EnergyName = '{}{}{}'.format("./",loc,"/Energy_Total.jpg")
EnergyNName = '{}{}{}'.format("./",loc,"/Energy_NBody.jpg")
EnergyUName = '{}{}{}'.format("./",loc,"/Energy_UP.jpg")
EnergyDName = '{}{}{}'.format("./",loc,"/Energy_Delta.jpg")
EnergyMName = '{}{}{}'.format("./",loc,"/Energy_Mech.jpg")
EnergyD2Name = '{}{}{}'.format("./",loc,"/Energy_Pct.jpg")

#==================================================================================================
## Analysis
#
# Raw ULDM Energy

# Total Energy from Integrator
egylist = np.load('{}{}'.format(loc, '/Outputs/egylist.npy'),allow_pickle=True) * EPre

# Energy Due to N Body Potential
egpcmlist =  np.load('{}{}'.format(loc, '/Outputs/egpcmMlist.npy'),allow_pickle=True) * EPre # NEW
egpcmlist2 = np.load('{}{}'.format(loc, '/Outputs/egpcmlist.npy'),allow_pickle=True) * EPre # OLD

# Energy Due to ULDM Self-Interaction
egpsilist = np.load('{}{}'.format(loc, '/Outputs/egpsilist.npy'),allow_pickle=True) * EPre

# Energy Due to Quantum Fun
ekandqlist = np.load('{}{}'.format(loc, '/Outputs/ekandqlist.npy'),allow_pickle=True) * EPre

mtotlist = np.load('{}{}'.format(loc, '/Outputs/masseslist.npy'),allow_pickle=True) * EPre

# Uniform Correction Performed in Integrator!

# Reconstruct NBody Energy (Real)
NBo, KS, PS = PyUL.NBodyEnergy(MassListSI,TMDataSI,EndNum, a, length_units)

# Reconstruct NBody Energy (Physical)
# NBo, KS, PS = PyUL.NBodyEnergy(MassListSI,TMDataSI,EndNum)

if Relative:
    K0 = KS[0] #  Need attention
    EUnit = '$E_k(0)$'
    
    if UVel != [0,0,0]:
        
        VRelSI = CB(np.linalg.norm(UVel),s_velocity_unit,'m/s','v')
        
        printU(f'Initial Relative Speed is {VRelSI:.3f} m/s','QW')
        
        K0 = 1/2*MassListSI[0]*(VRelSI)**2

else: 
    K0 = 1
    EUnit = 'J'
# Processed ULDM Energy

MES = (PS + KS)
MESD = PyUL.GetRel(MES)/K0

EKQD = PyUL.GetRel(ekandqlist)/K0
EGPD = PyUL.GetRel(egpsilist)/K0

ECMD = PyUL.GetRel(egpcmlist)/K0
ECOD = PyUL.GetRel(egpcmlist2)/K0

KSD = PyUL.GetRel(KS)/K0
PSD = PyUL.GetRel(PS)/K0

EUOld  = egylist
EUOldD = PyUL.GetRel(EUOld)/K0

EUNew  = egpsilist + ekandqlist + egpcmlist2
EUNewD = PyUL.GetRel(EUNew)/K0

ETOld  = EUOld + MES
ETOldD = PyUL.GetRel(ETOld)/K0

ETNew  = EUNew + MES
ETNewD = PyUL.GetRel(ETNew)/K0


EROld = ETOld / ETOld[0]

ERNew = ETNew / ETNew[0]


#==================================================================================================
## Compile Printing Boxes

SimInfo = '\n'.join((
    TimeStamp,
    r'Resolution: $%.0f^3$' % (resol, ),
    r'Box Length: %.3f kpc' % (lengthKpc, ),
    r'Simulation Time Length: %.3f Myr' % (durationMyr, )
    ))

NBodyInfo = '\n'.join((
    TimeStamp,
    r'Number of Bodies: $%.0f$' % (NBo, ),
    ))

print(NBodyInfo)

##==================================================================================================
### Plots
#==================================================================================================
# Energy Change Diagram
fig = plt.figure(figsize = EFigSize)
ax = fig.add_subplot(111)

ax.plot(Tp,ECMD,EUStyle,color = EGColor,label='$E_{GP}$ (ULD Potential Due to Particles)')
ax.plot(Tp,EGPD,EUStyle,color = ERColor,label='$E_{GP}$ (ULD Potential Due to Self-Interaction)')
ax.plot(Tp,EKQD,EUStyle,color = EQColor,label='ULD $E_{K}+E_{Q}$')

ax.plot(Tp,EUNewD,ETStyle,color = EUColor,label='Total ULDM Energy')

if len(particles) >= 2:
    ax.plot(Tp,KSD,ENStyle,color = EKColor,label = '$\sum E_k$ of Particles')
    ax.plot(Tp,PSD,ENStyle,color = EPColor,label = '$\sum E_p$ of Particles')
    ax.plot(Tp,MESD,ETStyle,color = ENColor,label = 'Total Mechanical Energy of Particles')
else:
    ax.plot(Tp,KSD,ENStyle,color = ENColor,label = 'Particle Energy')
ax.plot(Tp,ETNewD,ETStyle,color = ETColor, label = 'Total Energy of System', lw = 5)
ax.set_ylabel(f'Change in Energy / {EUnit}')

ax.legend(ncol=3,bbox_to_anchor=(0.5, -0.4),loc='lower center')

plt.xlabel('Time / Myr')
plt.title('Energy Change of System')
plt.savefig(EnergyName, format='jpg', dpi=72)

plt.show()


#==================================================================================================
# Energy Change Diagram Distilled
fig = plt.figure(figsize = EFigSize)
ax = fig.add_subplot(111)

ax.plot(Tp,EUNewD,ETStyle,color = EUColor,label='Total ULDM Energy')

if len(particles) >= 2:
    ax.plot(Tp,KSD,ENStyle,color = EKColor,label = '$\sum E_k$ of Particles')
    ax.plot(Tp,PSD,ENStyle,color = EPColor,label = '$\sum E_p$ of Particles')
    ax.plot(Tp,MESD,ETStyle,color = ENColor,label = 'Total Mechanical Energy of Particles')
else:
    ax.plot(Tp,KSD,ENStyle,color = ENColor,label = 'Particle Energy')
ax.plot(Tp,ETNewD,ETStyle,color = ETColor, label = 'Total Energy of System',lw = 5)
ax.set_ylabel(f'Change in Energy / {EUnit}')

ax.legend(ncol=3,bbox_to_anchor=(0.5, -0.4),loc='lower center')

plt.xlabel('Time / Myr')


plt.title('Energy Change of System')
plt.savefig(EnergyDName, format='jpg', dpi=72)

plt.show()

#==================================================================================================
# Energy Change Diagram Distilled Further
fig = plt.figure(figsize = EFigSize)
ax = fig.add_subplot(111)

if len(particles) >= 2:
    ax.plot(Tp,KSD,ENStyle,color = EKColor,label = '$\sum E_k$ of Particles')
    ax.plot(Tp,PSD,ENStyle,color = EPColor,label = '$\sum E_p$ of Particles')
    ax.plot(Tp,MESD,ETStyle,color = ENColor,label = 'Total Mechanical Energy of Particles')
else:
    ax.plot(Tp,KSD,ENStyle,color = ENColor,label = 'Particle Energy')

ax.set_ylabel(f'Change in Energy / {EUnit}')

# ax.plot(Tp, - egpcmlist + egpcmlist[0] ,label = 'ULDM Potential Experienced by Particle')
# ax.plot(Tp, - egpcmlist2 + egpcmlist2[0],label = 'NBody Potential Experienced by ULDM')

ax.legend(ncol=3,bbox_to_anchor=(0.5, -0.4),loc='lower center')

plt.xlabel('Time / Myr')


plt.title('Energy Change of Particle System')
plt.savefig(EnergyNName, format='jpg', dpi=72)

plt.show()
#==================================================================================================
# The Advanced Energy Change Ratio Diagram
fig = plt.figure(figsize= EFigSize_Mono)
ax = fig.add_subplot(111)
#InfoBoxD = AnchoredText(SimInfo, loc=3)
#ax.add_artist(InfoBoxD)

ax.plot(Tp,ERNew,ETStyle, color = ETColor, label = 'Total Energy of Entire System')
ax.set_xlabel('Time / Myr')
ax.legend()

ax.set_ylabel('$ΔE/E$')

plt.savefig(EnergyD2Name, format='jpg', dpi=72)

plt.show()

##==================================================================================================
## The Advanced Energy Change Ratio Diagram 2 
#fig = plt.figure(figsize= EFigSize_Mono)
#ax = fig.add_subplot(111)
#
#ax.plot(Tp,TotalER2,ETStyle, color = ETColor, label = 'Total Energy of Entire System')
#ax.set_xlabel('Time / Myr')
#ax.legend()
#
#ax.set_ylabel('Relative Energy Change (%)')
##ax.set_ylim([90.0,110.0])
#ax.set_title('Percentage Change of System Energy (NEW)')
#
#plt.show()
#
#==================================================================================================
# The Net Mass Diagram

MassData = np.load('{}{}'.format(loc, '/Outputs/ULDMass.npy'),allow_pickle=True)

MDataMSol = ToPhys(MassData,'M_solar_masses','m')

fig = plt.figure(figsize= EFigSize_Mono)
ax = fig.add_subplot(111)

ax.plot(Tp,MDataMSol)
ax.set_xlabel('Time / Myr')

ax.set_ylabel('M / (Million Solar Masses)')
ax.set_title('Mass of ULDM in Box')
plt.show()

#==================================================================================================
# Energy Change Diagram Distilled Extremely

fig = plt.figure(figsize = EFigSize)
ax = fig.add_subplot(211)

ax.plot(Tp, egpcmlist,label = 'ULDM Potential Experienced by Particle')
ax.plot(Tp, egpcmlist2,label = 'NBody Potential Experienced by ULDM')

ax.legend()

plt.ylabel('Energy / J')

ax.set_title('Potential Energies')

plt.savefig(EnergyUName, format='jpg', dpi=72)

#==================================================================================================
# Redundancy Check 2

axR = fig.add_subplot(212)

GDiff = egpcmlist2 - egpcmlist

axR.plot(GDiff,'--',label= 'Difference between the two quantities.')
axR.legend()
axR.set_xlabel('Integration Snapshot Number')
plt.ylabel('Difference in Energy / J')


plt.show()

Entropy = np.load('{}{}'.format(loc, '/Outputs/Entro.npy'),allow_pickle=True) 
fig = plt.figure(figsize= EFigSize_Mono)
plt.plot(Tp,Entropy)

plt.ylabel('Entropy / ??')
plt.xlabel('Time / Myr')

## [Perturbative Treatment Comparison (QuantumWind)]

### Base

In [ ]:
from scipy.special import hyp1f1, gamma, factorial
import math
import time
import pyfftw
import os
import sys
import multiprocessing
from scipy.special import gamma

xLoc = PyUL.convert_between(TMDataSI[:,1],'m',length_units,'l') # The fixed Y Position

print(UVel, s_velocity_unit)

# Let's work everything in consistent astrophysical Units!.

Mass = MassListSI[0]

print(f'BH Mass is {Mass:.4g} kg')

vRel0 = -CB(UVel[1],s_velocity_unit,'m/s','v') + TMDataSI[:,4]

speed = vRel0[0] # m/s

lamb = 2*np.pi*PyUL.hbar/(PyUL.axion_mass*speed) # Full Quantum de Broglie

beta = 2*np.pi*(PyUL.G*Mass)/(speed**2*lamb) # Checked

print(f"de Broglie wavelength: {lamb:.5g} m\n\
Dimensionless Beta: {beta:.5g}")

lambKpc = PyUL.convert_between(lamb,'m',length_units,'l')

bound = length # Corresponding to the length we got. in User Units

resolA = 500

# Grid System for Analytical Result (Scaled Up by pi)

boundHyp = bound / 4

GridVec = np.linspace(-boundHyp*3,boundHyp*3,resolA,endpoint = False) 
GridVecL = np.linspace(-boundHyp*10,boundHyp*4,resolA,endpoint = False)

GridVecDisp = np.linspace(-bound,bound,resolA,endpoint = False) 
GridVecLDisp = np.linspace(-bound*3,bound,resolA*2,endpoint = False)

# Grid System for Simulations
GridVecR = np.linspace(-bound/2,bound/2,resol,endpoint = False)


vA,hA = np.meshgrid(GridVec,GridVecL,indexing = 'ij', sparse = True)

RArray = ne.evaluate('sqrt(vA*vA+hA*hA)')

SumLim = 172 # 172 is the maximum we can handle

ACom = beta*1j

BCom = 1

In = 1j*(RArray+hA)/lambKpc

Hyp = np.complex256(0)

for ii in range(SumLim):
    Hyp += gamma(ACom+ii)/gamma(ACom)/gamma(ii+1)**2*In**ii
    
    if ii == SumLim-1:
        print(gamma(ACom+ii)/gamma(ACom)/gamma(ii+1))
    
Hyp *= np.abs(gamma(1-beta*1j))

Hyp *= (np.exp(-1j*2*np.pi*hA/lambKpc + np.pi*beta/2))

Hyp *=  np.conj(Hyp)

plt.imshow(Hyp.real,cmap = Palette,aspect = 'auto')

In [ ]:
PlotB = False
PlotLD = True
# Plotting Simulated Data

print(EndNum)
I = -1

PsiR = (data[I]/DensityC)-1

TML = xLoc[I]

# Motion due to Particle Movement

TpSec = CB(Tp,'Myr','s','t')

dSim = TMDataSI[:,1] - TMDataSI[0,1]

vQW = -1*CB(UVel[1],s_velocity_unit,'m/s','v')

dQW = vQW*TpSec

bSI = dQW + dSim

bVKpc = CB(bSI,'m','kpc','l')

vRel0 = -CB(UVel[1],s_velocity_unit,'m/s','v') + TMDataSI[:,4]

mBH = MassList_MSol[0] # Million MSols
mBH_kg = CB(mBH,'M_solar_masses','kg','m')

l = 2*np.pi*PyUL.hbar/(PyUL.axion_mass*vRel0)

lRkPC = PyUL.convert_between(l,'m','kpc','l')

In [ ]:
PsiR = np.log10(np.real(data[I])/DensityC+1)

levels = np.linspace(-np.max(PsiR), np.max(PsiR), 300)

CSx = np.linspace(-bound-2*TML,bound-2*TML,resol,endpoint = False)
CSy = np.linspace(-bound,bound,resol,endpoint = False)
CAx = GridVecL/np.pi
CAy = GridVec/np.pi

BaseGrid = plt.GridSpec(2, 1, hspace=0)

figC = plt.figure(figsize= (8,6), dpi = 300)

axSim = figC.add_subplot(BaseGrid[0,0])
axAna = figC.add_subplot(BaseGrid[1,0],sharex = axSim)


axSim.set_aspect('equal')
axAna.set_aspect('equal')


axAna.margins(0.0)
axSim.margins(0.0)

circle = plt.Circle((0, 0), lRkPC[I], clip_on=True,fill = False,color = EmphColor,lw = 4, zorder = 100)
circleb = plt.Circle((0, 0), bVKpc[I], clip_on=True,fill = False,color = GeomColor,lw = 2, zorder = 100)

axSim.scatter([0],[0],color = EmphColor)

if PlotLD:
    axSim.add_patch(circle)
if PlotB:
    axSim.add_patch(circleb)

    
axSim.contour(CSx,CSy,PsiR, levels=levels,cmap = DivPalette)

axSim.tick_params(direction="in",length = 10)

axSim.tick_params(width = 2)
plt.setp(axSim.get_xticklabels(), visible=False)

Hypr = np.log10(np.abs(Hyp))

Mappable = axAna.contour(CAx,CAy,Hypr, levels=levels,cmap = DivPalette)


circle = plt.Circle((0, 0), lRkPC[I], clip_on=True,fill = False,color = EmphColor,lw = 4, zorder = 100)
circleb = plt.Circle((0, 0), bVKpc[I], clip_on=True,fill = False,color = GeomColor,lw = 2, zorder = 100)

if PlotLD:
    axAna.add_patch(circle)
if PlotB:
    axAna.add_patch(circleb)

axAna.scatter([0],[0],color = EmphColor)

axAna.tick_params(direction="in",length = 10)

axAna.tick_params(width = 2)

axAna.set_xlim(GridVecL[0]/np.pi,GridVecL[-1]/np.pi)

axAna.set_ylim(GridVec[0]/np.pi,GridVec[-1]/np.pi)

axSim.set_xlim(GridVecL[0]/np.pi,GridVecL[-1]/np.pi)

axSim.set_ylim(GridVec[0]/np.pi,GridVec[-1]/np.pi)

#cb = figC.colorbar(Mappable,cax = axCB,orientation = 'horizontal',ticks = [])
# figC.suptitle('{} {}{}'.format(TITLETEXT, plot_axis, '=0'), fontsize = 18)
#axSim.set_title('Simulation',fontsize=15)
#axSim.set_ylabel('y / kpc')
#
axAna.text(0.98,0.9,s='Coulomb Scattering',horizontalalignment='right',verticalalignment='center', transform=axAna.transAxes)
axSim.text(0.98,0.9,s='Simulation',horizontalalignment='right',verticalalignment='center', transform=axSim.transAxes)
axSim.set_yticks([])
axAna.set_yticks([])
# 

plt.show()

In [ ]:
PsiR = np.real(data[I]/DensityC-1)

BaseGrid = plt.GridSpec(2, 1, hspace=0,left=0,right=1.2)

figC = plt.figure(figsize= (8,6), dpi = 300)

axSim = figC.add_subplot(BaseGrid[0,0])
axAna = figC.add_subplot(BaseGrid[1,0],sharex = axSim)

axSim.set_aspect('equal')
axAna.set_aspect('equal')

circle = plt.Circle((0, 0), lRkPC[I], clip_on=True,fill = False,color = EmphColor,lw = 4)
circleb = plt.Circle((0, 0), bVKpc[I], clip_on=True,fill = False,color = GeomColor,lw = 2)

axSim.scatter([0],[0],color = EmphColor)

if PlotLD:
    axSim.add_patch(circle)
if PlotB:
    axSim.add_patch(circleb)

axSim.imshow((PsiR),
           origin = 'lower',
          cmap = DivPalette,
          extent = (-bound-2*TML,bound-2*TML,-bound,bound),
          vmin = -np.max(PsiR),
          vmax = np.max(PsiR))

axSim.tick_params(direction="in",length = 10)

axSim.tick_params(width = 2)
plt.setp(axSim.get_xticklabels(), visible=False)
Hypr = (np.abs(Hyp))-1

Mappable = axAna.imshow((Hypr),
           origin = 'lower',
          cmap = DivPalette,
          extent = (GridVecL[0]/np.pi,GridVecL[-1]/np.pi,GridVec[0]/np.pi,GridVec[-1]/np.pi),
          vmin = -np.max(PsiR),
          vmax = np.max(PsiR))


circle = plt.Circle((0, 0), lRkPC[I], clip_on=True,fill = False,color = EmphColor,lw = 4)
circleb = plt.Circle((0, 0), bVKpc[I], clip_on=True,fill = False,color = GeomColor,lw = 2)

if PlotLD:
    axAna.add_patch(circle)
if PlotB:
    axAna.add_patch(circleb)

axAna.scatter([0],[0],color = EmphColor)

axAna.tick_params(direction="in",length = 10)

axAna.tick_params(width = 2)

axAna.set_xlim(GridVecL[0]/np.pi,GridVecL[-1]/np.pi)

axAna.set_ylim(GridVec[0]/np.pi,GridVec[-1]/np.pi)

axSim.set_xlim(GridVecL[0]/np.pi,GridVecL[-1]/np.pi)

axSim.set_ylim(GridVec[0]/np.pi,GridVec[-1]/np.pi)

#cb = figC.colorbar(Mappable,cax = axCB,orientation = 'horizontal',ticks = [])
# figC.suptitle('{} {}{}'.format(TITLETEXT, plot_axis, '=0'), fontsize = 18)
#axSim.set_title('Simulation',fontsize=15)
#axSim.set_ylabel('y / kpc')
#
axAna.text(0.98,0.9,s='Coulomb Scattering',horizontalalignment='right',verticalalignment='center', transform=axAna.transAxes)
axSim.text(0.98,0.9,s='Simulation',horizontalalignment='right',verticalalignment='center', transform=axSim.transAxes)

axSim.set_yticks([])
axAna.set_yticks([])
# 

plt.show()

print(Tp[I])

### Animated Density Comparison 2D

In [ ]:
# Prepare the B Size

Hypr = np.log(np.abs(Hyp))

try:
    VTimeStamp = TimeStamp
except NameError:
    VTimeStamp = str('Debug')

AnimName = '{}{}{}{}'.format(loc,"/AnimDensityCompare_",VTimeStamp,".mp4")

print("Saving ",AnimName)

plot_axis = 'z' #axis that is set to zero
plt.ioff()

figC, (axSim, axAna) = plt.subplots(1, 2, figsize= VideoSize, dpi = DPI, 
                                    subplot_kw={'aspect': 'equal',
                                                'xlabel':'x / kpc'})
axCB = figC.add_subplot(10,1,10)

TITLETEXT = VTimeStamp + ': 2D Density Comparison'


# Frame Number Management
if FrameCap > 0 and FrameCap < EndNum:
    Step = EndNum//FrameCap
    MovEnd = FrameCap
else: 
    Step = 1
    MovEnd = EndNum

def animateC(i):

    I = i*Step
    
    axAna.cla()
    axSim.cla()
    axCB.cla()

    ts = time.time()

    # Plotting Simulated Data

    DataSlice = I
    
    circle1 = plt.Circle((0, 0), lRkPC[DataSlice], clip_on=True,fill = False,color = EmphColor,lw = 4)
    circle2 = plt.Circle((0, 0), lRkPC[DataSlice], clip_on=True,fill = False,color = EmphColor,lw = 4)

    PsiR = np.log10(data[DataSlice]/Density)

    TML = xLoc[DataSlice]

    circleb1 = plt.Circle((0, 0), bVKpc[I], clip_on=True,fill = False,color = GeomColor,lw = 2)
    circleb2 = plt.Circle((0, 0), bVKpc[I], clip_on=True,fill = False,color = GeomColor,lw = 2)
    
    axSim.scatter([0],[0],color = EmphColor)
    axSim.add_patch(circle1)
    axSim.add_patch(circleb1)

    axSim.set_ylabel('y / kpc')
    
    mappable = axSim.imshow(np.log(PsiR+1),
               origin = 'lower',
              cmap = DivPalette,
              extent = (-bound-2*TML,bound-2*TML,-bound,bound),
              vmin = -np.max(np.log(PsiR+1)),
              vmax = np.max(np.log(PsiR+1)))

    axSim.tick_params(direction="in",length = 10,width = 2)



    #cb = plt.colorbar(orientation = 'horizontal',ticks = [-0.0002,0,0.0002])
    #cb.ax.set_xticklabels(['Underdensity','Background','Overdensity'])
    #cb.ax.tick_params(direction="in",length = 0)
    cb = figC.colorbar(mappable,cax = axCB,orientation = 'horizontal', shrink = 0.7)

    figC.suptitle('{} {}{}'.format(TITLETEXT, plot_axis, '=0'), fontsize = 15)
    axSim.set_title('{}{:.4f}{}'.format('Simulation @', Tp[I],' Million Years'),fontsize=12)
    
    axAna.set_title('Coulomb Scattering Ref.',fontsize=12)
    
    mappableA = axAna.imshow((Hypr),
               origin = 'lower',
              cmap = DivPalette,
              extent = (GridVecL[0]/np.pi,GridVecL[-1]/np.pi,GridVec[0]/np.pi,GridVec[-1]/np.pi),
              vmin = -np.max(np.log(PsiR+1)),
              vmax = np.max(np.log(PsiR+1)))

    axAna.add_patch(circle2)
    axAna.add_patch(circleb2)

    axAna.scatter([0],[0],color = EmphColor)

    axAna.tick_params(direction="in",length = 10,width = 2)

    axAna.set_xlim(GridVecL[0]/2/np.pi,GridVecL[-1]/2/np.pi)
    axAna.set_ylim(GridVec[0]/2/np.pi,GridVec[-1]/2/np.pi)
    
    axSim.set_xlim(GridVecL[0]/2/np.pi,GridVecL[-1]/2/np.pi)
    axSim.set_ylim(GridVec[0]/2/np.pi,GridVec[-1]/2/np.pi)

    PyUL.prog_bar(MovEnd, i+1, time.time()-ts)
    if i == EndNum-1:
        print('\nAnimation Complete')
    return mappable,mappableA
interval = 0.001 #in seconds
aniC = matplotlib.animation.FuncAnimation(figC,animateC,MovEnd,interval=interval*1e+3,blit=True,)

Writer = matplotlib.animation.writers['ffmpeg']

writer = Writer(fps=FPS, metadata=dict(artist='PyUltraLightF'))

aniC.save(AnimName, writer=writer)

Video(AnimName)


### 1D Density Evo

In [ ]:
SliceName = '{}{}{}'.format("./",loc,"/Density1DE.jpg")

fig = plt.figure(figsize = (10,10), dpi = 300)
ax = fig.add_subplot(10, 1,  (2,10))

Hypr = (np.abs(Hyp)) - 1


Plottable = np.real(linedata)/DensityC - 1

Mappable = ax.imshow(Plottable, origin = 'lower', cmap = DivPalette,
                 extent = (-bound-2*TML,bound-2*TML,0,Tp[-1]),
                aspect = 'auto', )
    

plt.scatter([TMDataS[:,1]]-TML,Tp,c = 'k',zorder = 100, alpha = 0.7, s = 1)
ax.set_xlabel('x / kpc')
ax.set_ylabel('Time / Myr')

#plt.gca().set_position([0, 0, 1, 1])
#plt.colorbar(mappable = Mappable, orientation = 'vertical', label = 'Overdensity')


ax2= fig.add_subplot(10, 1,  1, yticks = [], sharex = ax)

ax2.imshow((Hypr),
           origin = 'lower',
          cmap = DivPalette,
          extent = (GridVecL[0]/np.pi/2,GridVecL[-1]/np.pi/2,GridVec[0]/np.pi/2,GridVec[-1]/np.pi/2),
          vmin = -np.max(Plottable),
          vmax = np.max(Plottable),
          aspect = 'auto')

ax2.set_ylim(GridVec[resolA//2-1],GridVec[resolA//2+1])
#ax2.set_xlim(GridVecL[0],GridVecL[-1])


plt.savefig(SliceName)

In [ ]:
plt.figure()

plt.plot(PlotRange-lengthKpc/4, Plottable[-1,:])
plt.plot(GridVecL/pi, Hypr[resolA//2,:])


### Horizontal Density Comparison

In [ ]:
Offset = 0

OffsetH = int(Offset*resolA*np.pi/resol)

fig, ax = plt.subplots(figsize = (16,4))

fig.patch.set_facecolor('white')
fig.patch.set_alpha(1)

ax.set_ylabel('Overdensity')
ax.set_xlabel('x / kpc')

print(Offset,OffsetH)

ax.plot(GridVecR-TML,PsiR[resol//2+Offset,:],'r-',label = f'PyUltraLight Simulation')

#ax.plot(GridVecR-TML,Coolline,'x',label = f'Simulated Line')

ax.plot(GridVecL/2/np.pi,Hypr[resolA//2+OffsetH,:],'k-', label = 'Analytical Solution')

ax.scatter([0],[0],marker = '.',color = 'k', s = 200)




ax.legend()

## [Dynamical Friction Workbench 2]


This section is for Use with ** Dynamical Friction ** presets only.

### Base

In [ ]:
vRel0 = -CB(UVel[1],s_velocity_unit,'m/s','v') + TMDataSI[:,4]

mBH = MassList_MSol[0] # Million MSols
mBH_kg = CB(mBH,'M_solar_masses','kg','m')


lR = PyUL.hbar/(PyUL.axion_mass*vRel0) # Has a factor of 2pi removed.
lRP = PyUL.convert_between(lR,'m','pc','l')

# Quantum Mach Number

MQ = 44.56*vRel0/1000*(((1e-22*PyUL.eV))/PyUL.axion_mass)/mBH/10

beta = 2*np.pi*PyUL.G*mBH_kg/(vRel0[0])**2/(lR[0])
betaTS = 2*np.pi*PyUL.G*mBH_kg/(vRel0)**2/(lR)
# Reference Force Value

FRel = 4*np.pi*Density*DPre*(PyUL.G*mBH*1e6*PyUL.solar_mass/(vRel0))**2


print(f'Initial Relative Speed: {vRel0[0]:.4f} m/s\n\
BH Mass: {mBH:.4f} Mil. MSol.')

print(f"λ0 = {2*np.pi*lRP[0]/1000:.6g} kpc")

print(f"M = {mBH_kg:.6g} kg")

print(f"β = {beta:.6g}")

print(f"Density = {Density:.6g} {density_unit}")

print(f'Reference Force: {FRel[0]:.5g} N')

### Evaluating b

In [ ]:
# Time in seconds
TpSec = CB(Tp,'Myr','s','t')

# Relative Distance Covered During Particle's Movement

bSI = np.cumsum(vRel0)*TpSec[1]
bSI[1:] = bSI[0:-1]
bSI[0] = 0
#bSI = dQW + dSim

bV = CB(bSI,'m','pc','l')

BVTilde = bV/lRP

vQW = -1*CB(UVel[1],s_velocity_unit,'m/s','v')

dQW = vQW*TpSec

bT = CB(dQW,'m','pc','l')

plt.figure(figsize = (10,8))

plt.xlabel('Time / Myr')
plt.ylabel("$b$ / kpc")

Lambda = BVTilde/beta

# Expected DynDrag

CLambda = np.log(2*Lambda) + 1/Lambda*np.log(2*Lambda) - 1


plt.plot(Tp,bV/1000, color = 'r',label = 'Simulated $b$')

plt.plot(Tp,bT/1000, '--', color = 'c', label = '$vt$')


plt.legend()

### Evaluating C

In [ ]:
# Gravitational Attraction of DM Wake
CLogDF2 = -1*np.array(graddata)[:,1] * PyUL.length_unit/PyUL.time_unit**2 # 1 Code Acceleration Unit

# Phi is already in m/s^2 

Crel = CLogDF2*vRel0**2/(4*np.pi*CB(Density,density_unit,'kg/m3','d')*PyUL.G**2*mBH_kg)
CrelMax = np.max(Crel)

# Exact Evaluation

from scipy.special import sici as SiCin


SbV,CbV = SiCin(2*BVTilde)

RealCbV = np.euler_gamma + np.log(2*BVTilde) - CbV

CrelV = RealCbV+np.sin(2*BVTilde)/(2*BVTilde)-1



fig = plt.figure(figsize = (10,6.18))
ax = fig.add_subplot(111)

ax.semilogy(Tp,Crel,color = 'red',label = f'PyUL Simulation')
ax.set_xlabel('Time / Myr')
ax.set_ylabel("$C$")
ax.plot(Tp,CrelV,'--',label = 'Coulomb Prediction', color = EmphColor)

plt.ylim(0.001,10)

### Main Framework

In [ ]:
beta

In [ ]:
plt.figure(figsize = (10,6.18))

BLine = np.linspace(-2,3,300)

BLineX = 10**BLine

rSys = PyUL.convert_between(length*BLineX,length_units,'m','l')

# Classical A
Lambda = BVTilde/betaTS

C_A = (1+Lambda)/(Lambda)*np.arctanh(np.sqrt(Lambda**2+2*Lambda)/(1+Lambda)) - np.sqrt(1+2/Lambda)

#plt.loglog(BVTilde,C_A,'-',          color = EmphColor, label = 'Classical Limit')


from scipy.special import sici as SiCin

Sb,Cb = SiCin(2*BLineX)

RealCb = np.euler_gamma + np.log(2*BLineX) - Cb

plt.loglog(BLineX,RealCb+np.sin(2*BLineX)/(2*BLineX)-1, '--',
           color = EmphColor, label = 'Coulomb Prediction')

#plt.loglog(BLineX,1/3*BLineX**2,color = 'green',label = 'Lancaster Low b Asymptote')

from scipy.special import digamma as Dig

beta = PyUL.G*mBH_kg/(vRel0[0]**2*lR[0])

#plt.loglog(BLineX,np.log(2*BLineX)-1-np.real(Dig(1+1j*beta)),color = 'orange',label = 'Lancaster High b Asymptote')


# Final A is already in m/s^2 
FinalA = CLogDF2


Crel = FinalA*vRel0**2/(4*np.pi*CB(Density,density_unit,'kg/m3','d')*PyUL.G**2*mBH_kg)

plt.scatter(BVTilde,Crel,label = 'PyUltraLight Simulation' ,  color = 'r', s = 35, marker = 'x' )
#plt.scatter(BITilde,Crel,label = 'No Time' ,  color = 'b', s = 35, marker = 'x' )

#plt.fill_betweenx(Crel,BCutTildeMin, BCutTildeMax,  color = 'r', alpha = 0.2)


plt.xticks([0.1,1,10,100,1000])
plt.yticks([0.1,1,10])

plt.xlabel('$\~{b} $')
plt.ylabel('$C$ ')

#plt.ylim(0.0005,100)

#plt.xlim(0.05,120)

plt.legend()

In [ ]:
PyUL.convert_back(1/a,'kpc','l')

In [ ]:
plt.plot(graddata)

In [ ]:
plt.imshow(data[0])

## 1D Mass Density

### Time Serialized Slice

In [ ]:
Step = 10
VOffset = 0.01
Sotal = EndNum // Step

PlotRange = np.linspace(-lengthKpc/2, lengthKpc/2,resol,endpoint = False)
plt.figure(figsize = (8,10))
for i in range(5,EndNum):
    if i%Step == 0:
        plt.plot(PlotRange,linedata[i]/DensityC+VOffset*i,'-',alpha = 0.7 + i/EndNum * 0.3)
        plt.scatter([TMDataS[i,1]],[1+VOffset*i],c = 'k',zorder = 100)
        
#plt.yticks([])

In [ ]:
SliceName = '{}{}{}'.format("./",loc,"/Density1D.jpg")

fig = plt.figure(figsize = (10,10), dpi = 300)
ax = fig.add_subplot(111)
ax.set_aspect('auto')

if DensityC == 0:
    
    Mappable = ax.imshow((np.real(linedata)), origin = 'lower', cmap = Palette,
                     extent = (-lengthKpc/2,lengthKpc/2,0,Tp[-1]),
                    aspect = 'auto')
else:
    print(DensityC)
    Plottable = np.real(linedata)/DensityC-1
    Mappable = ax.imshow(Plottable, origin = 'lower', cmap = DivPalette,
                     extent = (-lengthKpc/2,lengthKpc/2,0,Tp[-1]),
                    aspect = 'auto', )
    

plt.scatter([TMDataS[:,1]],Tp,c = 'k',zorder = 100, alpha = 0.7, s = 1)
ax.set_xlabel('x / kpc')
ax.set_ylabel('Time / Myr')

plt.gca().set_position([0, 0, 1, 1])
plt.colorbar(mappable = Mappable, orientation = 'vertical', label = 'Overdensity')


plt.savefig(SliceName)

## 1D Gravitational Field

In [ ]:
SliceName = '{}{}{}'.format("./",loc,"/GField1D_Total.jpg")

fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(111)
ax.set_aspect('auto')
Mappable = ax.imshow(phiF1D, origin = 'lower', cmap = Palette,
                     extent = (-lengthKpc/2,lengthKpc/2,0,Tp[-1]),
                    aspect = 'auto')

#for i,T in enumerate(Tp):
#    if i % 10 == 0:
#        plt.scatter([TMDataS[i,1]],[T],c = 'k',zorder = 100, alpha = 0.7)
        
ax.set_xlabel('x / kpc')
ax.set_ylabel('Time / Myr')

plt.gca().set_position([0, 0, 1, 1])
plt.colorbar(mappable = Mappable, orientation = 'vertical', label = 'Gravitational Field Strength')

#plt.xlim(3,6)

plt.savefig(SliceName)

### Animated Field

In [ ]:
try:
    VTimeStamp = TimeStamp
except NameError:
    VTimeStamp = str('Debug')

AnimName = '{}{}{}{}'.format(loc,"/AnimLineField_",VTimeStamp,".mp4")

figD = plt.figure(figsize = EFigSize, dpi = DPI)
axD = figD.add_subplot(111)

PlotRange = np.linspace(-lengthKpc/2, lengthKpc/2,resol,endpoint = False)

def animateD(i):
    
    ts = time.time()
    axD.cla()

    axD.plot(PlotRange,phi1D[i])
    axD.plot(PlotRange,phiF1D[i])
    axD.scatter(TMDataS[i,1],[0])
    axD.set_ylim(0,np.min(phiF1D))
    plt.ylabel('Gravitational Field')
    plt.xlabel('Position / kpc')
        
    PyUL.prog_bar(EndNum, i+1, time.time()-ts)
    if i == EndNum-1:
        print('\nAnimation Complete')
    

interval = 0.001 #in seconds
aniD = matplotlib.animation.FuncAnimation(figD,animateD,EndNum,interval=interval*1e+3,blit=False)

Writer = matplotlib.animation.writers['ffmpeg']

writer = Writer(fps=FPS, metadata=dict(artist='PyUltraLightF'))

aniD.save(AnimName, writer=writer)

Video(AnimName)

## 2D Complex Field

In [ ]:
try:
    VTimeStamp = TimeStamp
except NameError:
    VTimeStamp = str('Debug')

Relative = False
    
Loga = True 

DensityGR = Density

loc = save_path + '/' + TimeStamp

AnimName = '{}{}{}{}'.format(loc,"/AnimComplex_",VTimeStamp,".mp4")

if Loga:
    
    if Relative:
        data0 = np.log(np.real(data)/DensityGR)
        print("Evaluating Change Ratio.")
        planemax = np.max(data0)
        planemin = np.min(data0)
        TITLETEXT = 'Logarithmic Over (Under) density in Plane'

    else:
        data0 = np.log(np.real(data))
        planemax = np.max(data0)
        planemin = -2
        TITLETEXT = 'Logarithmic Mass Density in Plane'
    
    print("Using Log Plot, the Contour Level Limits Are")
    
    
else:
    if Relative:
        data0 = np.real(data)/DensityGR
        print("Initial Field is Uniform. Evaluating Change Ratio.")
        planemax = np.max(data0)
        planemin = np.min(data0)
        TITLETEXT = 'Over (Under) Density in Plane'

    else:
        data0 = np.real(data)
        planemax = np.max(data0)
        planemin = np.min(data0)
        TITLETEXT = 'Mass Density in Plane'

TITLETEXT = VTimeStamp + ': ' + TITLETEXT

print("Saving ",AnimName)

plot_axis = 'z' #axis that is set to zero
plt.ioff()
fig0, ax0 = plt.subplots(figsize=VideoSize, dpi=DPI)
ax0.set_aspect('equal')

PlotRange = np.linspace(-lengthKpc/2, lengthKpc/2,resol,endpoint = False)

# print(PlotRange)

levels = np.linspace(planemin, planemax, int(resol))

# Frame Number Management
if FrameCap > 0 and FrameCap < EndNum:
    Step = EndNum//FrameCap
    MovEnd = FrameCap
else: 
    Step = 1
    MovEnd = EndNum
    
# With out-of-bounds colors:

norm = mpl.colors.BoundaryNorm(boundaries=levels, ncolors=300, extend='both')

def animate3(i):
    I = Step * i
    ts = time.time()
    
    ax0.cla()
    
    ax0.set_aspect('equal')
    ax0.get_xaxis().set_ticks([])
    ax0.get_yaxis().set_ticks([])
    
    ax0.set_xlim([-lengthKpc/2,lengthKpc/2])
    ax0.set_ylim([-lengthKpc/2,lengthKpc/2])
    
    Plane = np.real(phasedata[I])
    
    ax0.imshow(Plane,cmap = CycPalette,origin = 'lower',
               vmin = -np.pi, vmax = np.pi,
               extent = (-lengthKpc/2,lengthKpc/2,-lengthKpc/2,lengthKpc/2),
              interpolation = 'none')
    
    TMStateLoc = TMDataS[I,:]
    
    for particleID in range(len(particles)):

        Color = Colors[np.mod(particleID,5)] # 0, 0.5
        
        TMx = TMStateLoc[int(6*particleID+1)]
        TMy = TMStateLoc[int(6*particleID)]
        TMz = TMStateLoc[int(6*particleID+2)]
        
        Vx = TMStateLoc[int(6*particleID+4)]
        Vy = TMStateLoc[int(6*particleID+3)]
        Vz = TMStateLoc[int(6*particleID+5)]
        ax0.plot([TMx],[TMy],'o',color=(Color[0],Color[1],Color[2],1))
        #ax0.quiver([TMx],[TMy],[Vx],[Vy])
        
    ax0.contour(PlotRange,PlotRange,data0[I], levels=levels,cmap = Palette)
    
    #ax0.pcolormesh(PlotRange, PlotRange,data0[I],rasterized = True, norm = norm, cmap = DivPalette)
    
    ax0.text(0.5, 1.05, '{}{:.4f}{}'.format('Time Elapsed: ', Tp[I],' Million Years'),
             horizontalalignment='center', verticalalignment='center', 
             transform=ax0.transAxes,color = EmphColor)
    
    
    ax0.text(0.5, -0.05, f'{ETNewD[I]}',
             horizontalalignment='center', verticalalignment='center', 
             transform=ax0.transAxes,color = EmphColor, fontsize = 15)
    fig0.suptitle('{} {}{}'.format(TITLETEXT, plot_axis, '=0'), fontsize = 12)
    PyUL.prog_bar(MovEnd, i+1, time.time()-ts)


interval = 0.15 #in seconds
ani3 = matplotlib.animation.FuncAnimation(fig0,animate3,MovEnd,interval=interval*1e+3,blit=False)

Writer = matplotlib.animation.writers['ffmpeg']

writer = Writer(fps=FPS, metadata=dict(artist='PyUltraLightF'))

ani3.save(AnimName, writer=writer)

Video(AnimName)

## 2D Mass Density

In [ ]:
Loga = False
Contour = True
Relative = False


try:
    VTimeStamp = TimeStamp
except NameError:
    VTimeStamp = str('Debug')

AnimName = '{}{}{}{}'.format(loc,"/AnimDensity_",VTimeStamp,".mp4")

print("Saving ",AnimName)

plot_axis = 'z' #axis that is set to zero
plt.ioff()
fig0, ax0 = plt.subplots(figsize=VideoSize, dpi=DPI)
ax0.set_aspect('equal')

Tp = np.linspace(0,EndNum-1,EndNum)
Tp = Tp * durationMyr / (save_number+1)

DensityGR = Density

if Loga:
    
    if Relative:
        data0 = np.log(np.real(data)/DensityGR)
        print("Initial Field is Uniform. Evaluating Change Ratio.")
        planemax = np.max(data0)
        planemin = np.min(data0)
        TITLETEXT = 'Logarithmic Over (Under) density in Plane'

    else:
        data0 = np.log(np.real(data))
        planemax = np.max(data0)
        planemin = -2
        TITLETEXT = 'Logarithmic Mass Density in Plane'
    
    print("Using Log Plot, the Contour Level Limits Are")
    
    
else:
    if Relative:
        data0 = np.real(data)/DensityGR
        print("Initial Field is Uniform. Evaluating Change Ratio.")
        planemax = np.max(data0)
        planemin = np.min(data0)
        TITLETEXT = 'Over (Under) Density in Plane'

    else:
        data0 = np.real(data)
        planemax = np.max(data0)
        planemin = np.min(data0)
        TITLETEXT = 'Mass Density in Plane'
        
TITLETEXT = VTimeStamp + ': ' + TITLETEXT

levels = np.linspace(planemin, planemax, int(resol/4))

print(f"Max:{planemax},Min:{planemin}")

PlotRange = np.linspace(-lengthKpc/2, lengthKpc/2,resol,endpoint = False)


# Frame Number Management
if FrameCap > 0 and FrameCap < EndNum:
    Step = EndNum//FrameCap
    MovEnd = FrameCap
else: 
    Step = 1
    MovEnd = EndNum
    
def animate0(i):
    
    I = Step * i
    
    ts = time.time()
    ax0.cla()
    ax0.set_aspect('equal')
    
    ax0.set_xticks([])
    ax0.set_yticks([])
    
    ax0.set_xlim([-lengthKpc/2,lengthKpc/2])
    ax0.set_ylim([-lengthKpc/2,lengthKpc/2])
    plt.setp(ax0.spines.values(), linewidth=0)
    
    if Contour:
        mappable = ax0.contour(PlotRange,PlotRange,data0[I], levels=levels,cmap = Palette)
    
    else:
        mappable = ax0.imshow(data0[I],cmap = Palette,origin = 'lower',
                  vmin = planemin, vmax = planemax, interpolation = 'bicubic',
                   extent = (-lengthKpc/2,lengthKpc/2,-lengthKpc/2,lengthKpc/2))
    

    TMStateLoc = TMDataS[I,:]
    
        
    TMx = TMStateLoc[1::6]
    TMy = TMStateLoc[0::6]
    #TMz = TMStateLoc[2::6]

    Vx = TMStateLoc[4::6]
    Vy = TMStateLoc[3::6]
    #Vz = TMStateLoc[5::6]
    
   
        
    ax0.scatter([TMx],[TMy],s = 30, color=EmphColor,)
    
    ax0.scatter([TMx[0]],[TMy[0]],s = 100, color='pink',)
        
    ax0.quiver([TMx[0]],[TMy[0]],[Vx[0]],[Vy[0]],color= 'red')
      
        
    #fig0.suptitle('{} {}{}'.format(TITLETEXT, plot_axis, '=0'), fontsize = 15)
    # ax0.text(0.5, 1.05, '{}{:.4f}{}'.format('Time Elapsed: ', Tp[I],' Million Years'), horizontalalignment='center', verticalalignment='center', transform=ax0.transAxes,color = 'orange')
    PyUL.prog_bar(MovEnd, i+1, time.time()-ts)

interval = 0.001 #in seconds
ani0 = matplotlib.animation.FuncAnimation(fig0,animate0,MovEnd,interval=interval*1e+3,blit=False)

Writer = matplotlib.animation.writers['ffmpeg']

writer = Writer(fps=FPS, metadata=dict(artist='PyUltraLightF'))

ani0.save(AnimName, writer=writer)

Video(AnimName)
